In [51]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

In [52]:
data = pd.read_parquet("data/full_qanun_text.parquet")

In [53]:
transformer = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-tas-b").to("cuda")

c:\Users\Rog\anaconda3\envs\NLP\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [59]:
batch_size = 1000
start = 0
indexes_list = []
len(data)

1501973

In [60]:
for i in tqdm(range(0, len(data), batch_size)):    
    if i <= len(data) - batch_size:
        dt = list(data.iloc[range(start, i + 1), :]["text"])
        dt = list(transformer.encode(dt))
        indexes_list += dt
        start += batch_size
    else:
        dt = list(data.iloc[i::, :]["text"])
        dt = list(transformer.encode(dt))
        indexes_list += dt
        start += batch_size

100%|██████████| 1502/1502 [00:09<00:00, 152.11it/s]


In [62]:
index = faiss.index_factory(768, 'IVF15,Flat')

In [64]:
index.train(np.array(indexes_list))

In [66]:
index.add(np.array(indexes_list))

In [67]:
faiss.write_index(index, "trained_index.index")